In [0]:
import pyspark.sql.functions as fcn
import pandas
from datetime import datetime, timedelta
from pyspark.sql.window import Window
import os
from marel.services import ServiceProvider
from marel.databricks import DatabricksProvider
provider = DatabricksProvider()

In [0]:
Months = 3
df_sx_telemetry = spark.table('marel_digital_machines.sensorx.marel_sensorx_telemetry')
df_sx_telemetry = df_sx_telemetry = df_sx_telemetry.filter(fcn.col('timeStamp') > fcn.add_months(fcn.current_date(),-Months)).select("timeStamp","properties_deviceId","payload_xrayController_filamentCurrent","payload_xrayController_temperature","payload_xrayController_tubeCurrent", "payload_xrayController_tubeVoltage", "payload_xrayController_onTime")

#display(df_sx_telemetry.limit(100))
display(df_sx_telemetry.count())

In [0]:
Months = 3
df_sx_serialnumber = (
    spark.table("marel_digital_machines.sensorx.marel_sensorx_xraycontroller_property_serialnumber")
    .filter(fcn.col("timeStamp") > fcn.add_months(fcn.current_date(), -Months))
    .groupBy("properties_deviceId", "payload_serialNumber")
    .agg(
        fcn.min("timeStamp").alias("min_timeStamp"),
        fcn.max("timeStamp").alias("max_timeStamp")
    )
)
display(df_sx_serialnumber.limit(100))

In [0]:
from pyspark.sql import functions as fcn

BIN_SIZE_SECONDS = 86400

T = df_sx_telemetry.alias("T")
S = df_sx_serialnumber.alias("S")

df_sx_telemetry_serialnumber = (
    T.hint("range_join", BIN_SIZE_SECONDS)
     .join(
        S,
        (fcn.col("T.properties_deviceId") == fcn.col("S.properties_deviceId")) &
        (fcn.col("T.timeStamp") >= fcn.col("S.min_timeStamp")) &
        (fcn.col("T.timeStamp") <= fcn.col("S.max_timeStamp")),
        how="left"
     )
     # Keep ALL telemetry columns + just serial from S
     .select("T.*", fcn.col("S.payload_serialNumber").alias("serialNumber"))
)

df_one = df_sx_telemetry_serialnumber.filter(fcn.col("properties_deviceId") == "08947f89-b3c1-4b1e-ed33-08d9f153eeaf")
display(df_one)


In [0]:
from pyspark.sql import functions as F, Window

Months = 1
df_sx_xraycontroller_fault = spark.table('marel_digital_machines.sensorx.marel_sensorx_xraycontroller_property_fault')
df_sx_xraycontroller_fault = df_sx_xraycontroller_fault.filter((fcn.col('timeStamp') > fcn.add_months(fcn.current_date(),-Months))).select("timeStamp","properties_deviceId","payload_fault_faultName","payload_fault_state")


tolerance_seconds = 30

m = df_sx_telemetry.alias("m")
l = df_sx_xraycontroller_fault.alias("l")

df_joined = (
    m.join(
        l,
        (F.col("m.properties_deviceId") == F.col("l.properties_deviceId")) &
        (F.abs(F.col("m.timeStamp").cast("long") - F.col("l.timeStamp").cast("long")) <= tolerance_seconds),
        "left"
    )
    .withColumn(
        "time_diff_seconds",
        F.abs(F.col("m.timeStamp").cast("long") - F.col("l.timeStamp").cast("long"))
    )
)

# How many rows actually matched?
matched = df_joined.filter(F.col("l.timeStamp").isNotNull()).count()
total = df_joined.count()
print("matched rows:", matched, "out of", total)

display(df_joined)
